In [1]:
from polygon import StocksClient, ReferenceClient

# from polygon.models import (
#     TickerSnapshot,
#     Agg,|
# )
import numpy as np
import streamlit as st
import yfinance as yf
import pandas as pd

In [ ]:
poly_ref_client = ReferenceClient(api_key="your_api_key")

In [3]:
tickers = []
for response in poly_ref_client.get_tickers(
    market="stocks", active=True, symbol_type="CS"
):
    # print(response["status"])
    tickers.append(response)

In [4]:
def get_active_exchange_stock_ticker(*exchanges):
    tickers = []
    api_call_count = 0
    for exchange in exchanges:
        print(f"Getting tickers for exchange {exchange}")
        url = None
        while True:
            api_call_count += 1
            if api_call_count > 5:
                print(
                    f"Warning: API call count has exceeded 5 Free Polygon has limit of 5 Calls Per Minute. Current count: {api_call_count}"
                )

            response = (
                poly_ref_client.get_page_by_url(url)
                if url
                else poly_ref_client.get_tickers(
                    market="stocks", active=True, symbol_type="CS", exchange=exchange
                )
            )
            print(
                f"Response Status: {response.get('status')},Count :{response.get('count')} Next URL: {response.get('next_url')}"
            )
            filtered_tickers = [
                i
                for i in response.get("results", [])
                if i["primary_exchange"] == exchange
            ]
            print(f"Filtered tickers count: {len(filtered_tickers)}")
            tickers.extend(filtered_tickers)
            if not (url := response.get("next_url")):
                break
    return tickers

In [5]:
# tickers = get_active_exchange_stock_ticker("XNYS", "XNAS")

tickers = get_active_exchange_stock_ticker("XNYS")

Getting tickers for exchange XNYS


Response Status: OK,Count :1000 Next URL: https://api.polygon.io/v3/reference/tickers?cursor=YWN0aXZlPXRydWUmYXA9TUFOJTdDOTMwOTA1OTcwNjU1MDJmOTFkNjA2MTlhNWJlMjZjMzBiN2YwMWQ4OWFhNjFjYzkxZThkYjdjMmUxN2ViMWMwYSZhcz0mZGF0ZT0yMDI1LTAzLTIzJmV4Y2hhbmdlPVhOWVMmbGltaXQ9MTAwMCZtYXJrZXQ9c3RvY2tzJm9yZGVyPWFzYyZzb3J0PXRpY2tlciZ0eXBlPUNT
Filtered tickers count: 1000
Response Status: OK,Count :769 Next URL: None
Filtered tickers count: 769


In [6]:
tickers[0]

{'ticker': 'A',
 'name': 'Agilent Technologies Inc.',
 'market': 'stocks',
 'locale': 'us',
 'primary_exchange': 'XNYS',
 'type': 'CS',
 'active': True,
 'currency_name': 'usd',
 'cik': '0001090872',
 'composite_figi': 'BBG000C2V3D6',
 'share_class_figi': 'BBG001SCTQY4',
 'last_updated_utc': '2025-02-27T00:00:00Z'}

In [7]:
from collections import Counter
counter = Counter(t["name"] for t in tickers)
duplicates = [name for name, count in counter.items() if count > 1]

# Print the duplicates
if duplicates:
    print(f"Duplicate names found: {duplicates}")
len(duplicates)

Duplicate names found: ['CMS Energy Corporation', 'Sun Life Financial Inc.', 'Block, Inc.', 'Trueblue, Inc.', 'Tenet Healthcare Corporation New', 'U-Haul Holding Company']


6

In [8]:
ticker_names = [t["ticker"] for t in tickers]

In [9]:
len(ticker_names)

1769

In [10]:
from itertools import islice
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
   pass

session = CachedLimiterSession(
   limiter=Limiter(RequestRate(100, Duration.SECOND)),
   bucket_class=MemoryQueueBucket,
   backend=SQLiteCache("yfinance.cache"),
)
def fetch_ticker_history(ticker_batch,period)->pd.DataFrame:
    """Fetch history for a single ticker."""
    tickers = yf.Tickers(ticker_batch, session=session)
    historical_data:pd.DataFrame =  tickers.history(period=period)
    return historical_data

def fetch_ticker_outstanding_shares(ticker_batch,period=30)->pd.DataFrame:
    shares_outstanding_dfs = []
    for ticker in ticker_batch:
        try:
            shares_outstanding = pd.DataFrame()
            shares_outstanding["shares"] = yf.Ticker(ticker).get_shares_full(pd.Timestamp.utcnow() - pd.Timedelta(days=period))
            shares_outstanding.index = pd.date_range(
                end=pd.Timestamp.utcnow().date(), periods=len(shares_outstanding), freq='D'
            )[::-1]
            shares_outstanding = shares_outstanding.rename_axis("Date").reset_index()
            shares_outstanding["Ticker"] = ticker
            shares_outstanding_dfs.append(shares_outstanding)
        except Exception as e:
            print(f"Error fetching shares outstanding for ticker {ticker}: {e}")
    data =  pd.concat(shares_outstanding_dfs, ignore_index=True)
    if data.empty:
        print(f"No data returned for ticker share: {ticker_batch}")
        return None
    return data

def fetch_ticker_history_with_name(ticker_batch, period="30d"):
    try:
        data = fetch_ticker_history(ticker_batch, period)
        if data.empty:
            print(f"No data returned for ticker batch: {ticker_batch}")
        return data
    except Exception as e:
        print(f"Error fetching data for ticker batch {ticker_batch}: {e}")
        return None

def chunked_iterable(iterable, size):
    it = iter(iterable)
    while chunk := list(islice(it, size)):
        yield chunk

In [11]:

from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor
import os
# Use ThreadPoolExecutor to fetch data concurrently
with ThreadPoolExecutor(max_workers=os.cpu_count()*2) as executor:
    res_stock = executor.map(fetch_ticker_history_with_name, chunked_iterable(ticker_names[:10], 100))
    res_shares = executor.map(fetch_ticker_outstanding_shares, chunked_iterable(ticker_names[:10], 100))
valid_results = [df for df in res_stock if df is not None and not df.empty]
if valid_results:
    ticker_stocks = pd.concat(valid_results, ignore_index=False)
else:
    print("No valid data to concat stocks")
    ticker_stocks = pd.DataFrame()

valid_results = [df for df in res_shares if df is not None and not df.empty]
if valid_results:
    ticker_shares = pd.concat(valid_results, ignore_index=False)
else:
    print("No valid data to concat shares")
    ticker_shares = pd.DataFrame()

[**********************80%*************          ]  8 of 10 completed404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=hEMU%2FQhHV10
[*********************100%***********************]  10 of 10 completed

1 Failed download:
['ABC']: AttributeError("'NoneType' object has no attribute 'update'")
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=hEMU%2FQhHV10


Error fetching shares outstanding for ticker ABC: 'NoneType' object has no attribute 'update'


In [12]:
ticker_shares_filtered = ticker_shares[ticker_shares["Ticker"] == "ABBV"]
print(ticker_shares_filtered)

          Date      shares Ticker
103 2025-03-23  1765350016   ABBV
104 2025-03-22  1775180032   ABBV
105 2025-03-21  1765350016   ABBV
106 2025-03-20  1765350016   ABBV
107 2025-03-19  1765350016   ABBV
108 2025-03-18  1765350016   ABBV
109 2025-03-17  1765350016   ABBV
110 2025-03-16  1765350016   ABBV
111 2025-03-15  1765350016   ABBV
112 2025-03-14  1765350016   ABBV
113 2025-03-13  1765350016   ABBV
114 2025-03-12  1765350016   ABBV
115 2025-03-11  1778659968   ABBV
116 2025-03-10  1765350016   ABBV
117 2025-03-09  1781750016   ABBV
118 2025-03-08  1765350016   ABBV


In [ ]:
# temp = ticker_stocks.reset_index()
# temp,temp["Close","A"]

(Price  index       Date Adj Close       Close                             \
 Ticker                        ABC           A         AA    AACT     AAM   
 0          0 2025-02-07       NaN  145.210007  36.004711  11.030  10.180   
 1          1 2025-02-10       NaN  143.690002  36.802155  11.040  10.160   
 2          2 2025-02-11       NaN  141.750000  37.051357  11.030  10.175   
 3          3 2025-02-12       NaN  138.080002  36.214043  11.030  10.170   
 4          4 2025-02-13       NaN  136.449997  36.263882  11.050  10.170   
 5          5 2025-02-14       NaN  134.669998  36.124329  11.045  10.160   
 6          6 2025-02-18       NaN  135.199997  36.891869  11.050  10.170   
 7          7 2025-02-19       NaN  137.149994  36.064522  11.050  10.170   
 8          8 2025-02-20       NaN  136.789993  37.131104  11.060  10.170   
 9          9 2025-02-21       NaN  135.369995  34.290203  11.060  10.170   
 10        10 2025-02-24       NaN  135.279999  34.908222  11.060  10.185   

In [18]:
ticker_stocks["Close"].T

Date,2025-02-07,2025-02-10,2025-02-11,2025-02-12,2025-02-13,2025-02-14,2025-02-18,2025-02-19,2025-02-20,2025-02-21,...,2025-03-10,2025-03-11,2025-03-12,2025-03-13,2025-03-14,2025-03-17,2025-03-18,2025-03-19,2025-03-20,2025-03-21
Ticker,,,,,,,,,,,,,,,,,,,,,
A,145.210007,143.690002,141.750000,138.080002,136.449997,134.669998,135.199997,137.149994,136.789993,135.369995,...,121.900002,121.029999,119.849998,117.330002,121.180000,122.489998,123.099998,122.209999,120.279999,120.750000
AA,36.004711,36.802155,37.051357,36.214043,36.263882,36.124329,36.891869,36.064522,37.131104,34.290203,...,30.980000,31.959999,33.250000,32.310001,33.529999,34.790001,33.970001,35.310001,34.840000,33.950001
AACT,11.030000,11.040000,11.030000,11.030000,11.050000,11.045000,11.050000,11.050000,11.060000,11.060000,...,11.100000,11.160000,11.140000,11.160000,11.150000,11.170000,11.170000,11.170000,11.190000,11.180000
AAM,10.180000,10.160000,10.175000,10.170000,10.170000,10.160000,10.170000,10.170000,10.170000,10.170000,...,10.215000,10.215000,10.230000,10.220000,10.230000,10.240000,10.240000,10.250000,10.250000,10.240000
AAMI,25.209459,25.059523,26.388968,25.189470,25.619289,25.679264,26.309000,26.219038,25.909168,25.699255,...,23.630121,23.939993,23.800049,23.920000,24.580000,25.080000,25.350000,25.980000,25.980000,25.530001
AAP,45.950001,45.240002,47.000000,47.490002,48.590000,47.869999,44.570000,44.320000,44.340000,43.439999,...,36.580002,36.520000,37.580002,38.139999,37.590000,37.840000,37.720001,36.919998,37.639999,37.459999
AAT,22.281214,22.172911,22.133526,21.680616,22.044914,21.936609,21.651079,21.375393,21.680616,21.493546,...,20.809999,20.389999,20.260000,19.650000,20.129999,20.370001,20.250000,20.709999,20.510000,20.250000
AB,35.816231,34.961147,34.669643,34.008900,34.931999,36.030003,36.090000,36.169998,36.130001,35.730000,...,36.779999,36.869999,36.869999,36.570000,37.070000,37.610001,37.799999,38.009998,38.119999,38.029999
ABBV,190.600006,190.339996,191.830002,193.000000,193.449997,192.869995,196.250000,197.350006,200.880005,202.080002,...,216.660004,213.589996,212.059998,211.550003,211.770004,214.470001,213.850006,212.250000,211.960007,210.009995


In [ ]:
# ticker_close =ticker_stocks["Close"]
# ticker_low = ticker_stocks["Low"]
# ticker_high = ticker_stocks["High"]
# ticker_volume = ticker_stocks["Volume"]
# ticker_open = ticker_stocks["Open"]
# ticker_close = ticker_close.reset_index().melt(id_vars=['Date'], var_name='Ticker', value_name='Price').reset_index(drop=True)
# ticker_low = ticker_low.reset_index().melt(id_vars=['Date'], var_name='Ticker', value_name='Low').reset_index(drop=True)
# ticker_high = ticker_high.reset_index().melt(id_vars=['Date'], var_name='Ticker', value_name='High').reset_index(drop=True)
# ticker_volume = ticker_volume.reset_index().melt(id_vars=['Date'], var_name='Ticker', value_name='Volume').reset_index(drop=True)
# ticker_open = ticker_open.reset_index().melt(id_vars=['Date'], var_name='Ticker', value_name='Open').reset_index(drop=True)
# ticker_df = pd.merge(ticker_close, ticker_low, on=['Date', 'Ticker'])
# ticker_df = pd.merge(ticker_df, ticker_high, on=['Date', 'Ticker'])
# ticker_df = pd.merge(ticker_df, ticker_volume, on=['Date', 'Ticker'])
# ticker_df = pd.merge(ticker_df, ticker_open, on=['Date', 'Ticker'])

In [19]:
cols =["Close","Low","High","Volume","Open"]
melted_dfs = []
for col in cols:
    melted_df = ticker_stocks[col].reset_index().melt(
        id_vars=['Date'], var_name='Ticker', value_name=col
    ).reset_index(drop=True)
    melted_dfs.append(melted_df)

# Merge all melted dataframes on 'Date' and 'Ticker'
from functools import reduce
ticker_df = reduce(lambda left, right: pd.merge(left, right, on=['Date', 'Ticker']), melted_dfs)

In [20]:
ticker_df

,Date,Ticker,Close,Low,High,Volume,Open
0,2025-02-07,A,145.210007,144.410004,149.389999,1557800.0,148.210007
1,2025-02-10,A,143.690002,141.830002,145.850006,1764700.0,145.580002
2,2025-02-11,A,141.750000,141.320007,144.199997,1134200.0,142.229996
3,2025-02-12,A,138.080002,136.399994,139.800003,2103600.0,138.779999
4,2025-02-13,A,136.449997,136.059998,138.350006,1395400.0,138.270004
...,...,...,...,...,...,...,...
295,2025-03-17,ABC,NaN,NaN,NaN,NaN,NaN
296,2025-03-18,ABC,NaN,NaN,NaN,NaN,NaN
297,2025-03-19,ABC,NaN,NaN,NaN,NaN,NaN
298,2025-03-20,ABC,NaN,NaN,NaN,NaN,NaN


In [35]:
stocks_df = pd.merge(ticker_shares, ticker_df, on=['Date', 'Ticker'], how='inner')
stocks_df["MCap"] = stocks_df["shares"] * stocks_df["Close"]

In [36]:
import duckdb
import pandas as pd


In [37]:
duck_conn = duckdb.connect('stocks.duckdb')

In [52]:
#CREATE TABLE IF NOT EXISTS
duck_conn.execute("CREATE OR REPLACE TABLE stocks AS SELECT * FROM stocks_df")

In [56]:
query = """SELECT 
    Date, 
    Ticker, 
    MCap, 
    Close
FROM 
    stocks
WHERE 
    Date >= CURRENT_DATE - INTERVAL 30 DAY
QUALIFY ROW_NUMBER() OVER (PARTITION BY Date ORDER BY MCap DESC) <= 2
ORDER BY 
    Date, 
    MCap DESC;"""

In [59]:
duck_conn.execute(f"""
CREATE TABLE IF NOT EXISTS TopMcap AS 
{query}
""")

In [81]:
mcap_df = duck_conn.execute("SELECT * FROM TopMcap").fetchdf()

In [73]:
index_query = """
WITH StockWeights AS (
    SELECT 
        Date,
        Close,
        COUNT(*) OVER (PARTITION BY Date) AS StockCount
    FROM 
        TopMCap
)
SELECT 
    Date,
    SUM(Close / StockCount) AS EqualWeightedIndex
FROM 
    StockWeights
GROUP BY 
    Date
ORDER BY 
    Date;"""

In [74]:
equal_weighted_index = duck_conn.execute(index_query).fetchdf()

In [ ]:
equal_weighted_index.head()

,Date,EqualWeightedIndex
0,2025-03-03,31.230000
1,2025-03-04,31.150000
2,2025-03-05,32.380001
3,2025-03-06,32.830002
4,2025-03-07,79.769999


In [78]:
import plotly.express as px

In [83]:
st.title("Equal-Weighted Index Dashboard")

# Plot the Equal-Weighted Index vs. Date
st.subheader("Equal-Weighted Index Price vs. Date")
fig = px.line(equal_weighted_index, x='Date', y='EqualWeightedIndex', title='Equal-Weighted Index Price Over Time')
st.plotly_chart(fig)

# Display stock composition for a selected date
st.subheader("Stock Composition on a Selected Date")
selected_date = st.date_input("Select a Date", value=equal_weighted_index['Date'].min())
selected_date_data = mcap_df[mcap_df['Date'] == pd.Timestamp(selected_date)]

if not selected_date_data.empty:
    st.write(f"Stock Composition for {selected_date}:")
    st.dataframe(selected_date_data[['Ticker', 'Close']])
else:
    st.write("No data available for the selected date.")

2025-03-23 18:59:18.077 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-23 18:59:18.079 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-23 18:59:18.080 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-23 18:59:18.081 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-23 18:59:18.121 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-23 18:59:18.122 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-23 18:59:18.123 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-23 18:59:18.124 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar